## Import packages and modules

In [36]:
from models.tda_models import TDA_PI34_Model, TDA_PI42_Model, VECTOR_STITCHING_PI_Model_34, VECTOR_STITCHING_PI_Model_42
from models.raw_models import Raw_Model, Dummy_Model
from data_preprocessing import process

import tensorflow as tf


In [37]:
# models

raw_model = Raw_Model() # cnn models working on raw images
tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images


In [38]:
# indices for 10 fold cross validation

indices = [
    ((32427, 32677), (36258, 36358))
]

indices100 = [
    ((8028, 8128), (2237, 2337)),
    ((31081, 31181), (47635, 47735)),
    ((51438, 51538), (24766, 24866)),
    ((32427, 32527), (36258, 36358)),
    ((0, 100), (17901, 18001)),
    ((49216, 49316), (13836, 13936)),
    ((51207, 51307), (100, 200)),
    ((48137, 48237), (57296, 57396)),
    ((1000, 1100), (59758, 59858)),
    ((29740, 29840), (17885, 17985))
]

indices250 = [
    ((8028, 8278), (2237, 2337)),
    ((31081, 31331), (47635, 47735)),
    ((51438, 51688), (24766, 24866)),
    ((32427, 32677), (36258, 36358)),
    ((0, 250), (17901, 18001)),
    ((49216, 49466), (13836, 13936)),
    ((51207, 51457), (100, 200)),
    ((48137, 48387), (57296, 57396)),
    ((1000, 1250), (59758, 59858)),
    ((29740, 29990), (17885, 17985))
]

indices500 = [
    ((8028, 8528), (2237, 2337)),
    ((31081, 31581), (47635, 47735)),
    ((51438, 51938), (24766, 24866)),
    ((32427, 32927), (36258, 36358)),
    ((0, 500), (17901, 18001)),
    ((49216, 49716), (13836, 13936)),
    ((51207, 51707), (100, 200)),
    ((48137, 48637), (57296, 57396)),
    ((1000, 1500), (59758, 59858)),
    ((29740, 30240), (17885, 17985))
]





In [39]:
(X, y), _ = tf.keras.datasets.mnist.load_data()

tda_history = []
raw_history = []
vector_stitching_history = []
j=0

for i in indices:
    data = process(data=(X,y), training_indices=i[0], testing_indices=i[1], dist_ratio=10)

    #TDA model
    #early stopping
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    tda_model_34.model.fit(data["X_tr_tda_34"], data["y_tr_tda_34"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_tda_good_34"], data["y_test"]), callbacks=[early_stopping], verbose=1)
    #print the best validation accuracy
    print("krok: ", j)
    print("Best validation accuracy for TDA model 34: \t", max(tda_model_34.model.history.history['val_accuracy']))
    tda_history.append(max(tda_model_34.model.history.history['val_accuracy']))


    # RAW model
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    raw_model.model.fit(data["X_tr"], data["y_tr"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_random"], data["y_test"]), verbose=1)
    print("Best validation accuracy for RAW model: \t", max(raw_model.model.history.history['val_accuracy']))
    raw_history.append(max(raw_model.model.history.history['val_accuracy']))



    #Vector stitching model
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
    vector_stitching_model_34.model.fit(data["X_tr_vector_stitching_34"], data["y_tr_vector_stitching_34"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_vector_stitching_good_34"], data["y_test"]), callbacks=[early_stopping], verbose=1)
    print("Best validation accuracy for VS model 34: \t", max(vector_stitching_model_34.model.history.history['val_accuracy']))
    vector_stitching_history.append(max(vector_stitching_model_34.model.history.history['val_accuracy']))


    #reload models
    raw_model = Raw_Model() # cnn models working on raw images
    tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
    vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images

    
    j=j+1

1/4
2/4
3/4
4/4
Epoch 1/20
250/250 [==============================] - 4s 10ms/step - loss: 2.2891 - accuracy: 0.2640 - val_loss: 2.4201 - val_accuracy: 0.1800
Epoch 2/20
250/250 [==============================] - 2s 8ms/step - loss: 1.5044 - accuracy: 0.5120 - val_loss: 2.0428 - val_accuracy: 0.2400
Epoch 3/20
250/250 [==============================] - 2s 8ms/step - loss: 0.9916 - accuracy: 0.6920 - val_loss: 2.0555 - val_accuracy: 0.2400
Epoch 4/20
250/250 [==============================] - 2s 8ms/step - loss: 0.9376 - accuracy: 0.7360 - val_loss: 1.7795 - val_accuracy: 0.4500
Epoch 5/20
250/250 [==============================] - 2s 9ms/step - loss: 0.6161 - accuracy: 0.8160 - val_loss: 1.9238 - val_accuracy: 0.4100
Epoch 6/20
250/250 [==============================] - 2s 8ms/step - loss: 0.4142 - accuracy: 0.8680 - val_loss: 1.5220 - val_accuracy: 0.5100
Epoch 7/20
250/250 [==============================] - 2s 8ms/step - loss: 0.4540 - accuracy: 0.8840 - val_loss: 1.8713 - val_accura

In [21]:

# print means of history
print("TDA mean: ", sum(tda_history)/len(tda_history))
print("RAW mean: ", sum(raw_history)/len(raw_history))
print("Vector stitching mean: ", sum(vector_stitching_history)/len(vector_stitching_history))


TDA mean:  0.6370000004768371
RAW mean:  0.5299999997019768
Vector stitching mean:  0.8149999976158142


In [22]:
print("tda model:")
for i in tda_history:
    print(i)
print("\nraw model:")
for i in raw_history:
    print(i)
print("\nvector stitching model:")
for i in vector_stitching_history:
    print(i)


tda model:
0.6299999952316284
0.6100000143051147
0.6700000166893005
0.6200000047683716
0.6600000262260437
0.6399999856948853
0.5199999809265137
0.6899999976158142
0.7099999785423279
0.6200000047683716

raw model:
0.6800000071525574
0.27000001072883606
0.7699999809265137
0.7400000095367432
0.75
0.3100000023841858
0.7799999713897705
0.4699999988079071
0.33000001311302185
0.20000000298023224

vector stitching model:
0.8899999856948853
0.800000011920929
0.75
0.8399999737739563
0.8799999952316284
0.6499999761581421
0.8100000023841858
0.75
0.9100000262260437
0.8700000047683716
